In [ ]:
# Importing required packages
import pandas as pd
import numpy as np

import os
import gc

In [ ]:
path = '/opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/processed/overall/offline/item_level/'

df = pd.DataFrame()
for item in os.listdir(path) :
    if 'metadata' not in item and '.' not in item:
        for month in os.listdir(path + item) :
            for file in os.listdir(path + item + '/' + month) :
                temp_df = pd.read_parquet(path + item + '/' + month + '/' + file)
                if 'sld_menu_itm_id' not in temp_df.columns :
                    temp_df['sld_menu_itm_id'] = int(item.split('=')[1])
                    
                df = pd.concat([df, temp_df], axis=0)
                
df.head()

In [ ]:
df.info()

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)

df.shape

In [ ]:
288312 + 323479

In [ ]:
df['new_qt'] = np.where(df['pos_itm_tot_qt'].isna(), df['quantity'], df['pos_itm_tot_qt'])
df['new_gc'] = np.where(df['gc'].isna(), df['num_trans'], df['gc'])

del df['pos_itm_tot_qt']
del df['quantity']
del df['gc']
del df['num_trans']

df.rename(columns={'new_qt':'quantity', 'new_gc':'num_trans'}, inplace=True)

In [ ]:
df.info()

In [ ]:
del df['__index_level_0__']

In [ ]:
def format_dtype(df, add_monthid=False) :
    
    dtype_dict = {'mcd_gbal_lcat_id_nu':'int',
                 'sld_menu_itm_id':'int',
                 'num_trans':'int',
                 'quantity':'int',
                 'WAP':'float',
                 'Mode_price':'float'}
    df_columns = list(df.columns.values)
    
    if 'pos_busn_dt' in df_columns :
        df['pos_busn_dt'] = pd.to_datetime(df['pos_busn_dt'])
           
    if add_monthid :
        df['monthid'] = df['pos_busn_dt'].dt.strftime('%Y%m')
           
    for ind, val in dtype_dict.items():
        if ind in df_columns :
            df[ind] = df[ind].astype(val)
           
    return df


In [ ]:
df = format_dtype(df, add_monthid=True)
df.info()

In [ ]:
# Storing results
op_path = '/opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/processed/overall/ungrouped/wap_mer/'

os.makedirs(op_path, exist_ok=True)
df.to_parquet(op_path, partition_cols=['sld_menu_itm_id', 'monthid'])

In [ ]:
# load check
ch_df = pd.read_parquet(op_path)

ch_df.shape

In [ ]:
ch_df.drop_duplicates()